# Video Game Recomendation System

## Business understanding

## Data understanding

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Dataset from kaggle on [here](https://www.kaggle.com/datasets/cynthiarempel/amazon-us-customer-reviews-dataset?select=amazon_reviews_us_Digital_Video_Games_v1_00.tsv)

In [2]:
df_raw = pd.read_csv('./data/Digital_Video_Games.tsv', sep='\t', header=0, error_bad_lines=False)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144724 entries, 0 to 144723
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        144724 non-null  object
 1   customer_id        144724 non-null  int64 
 2   review_id          144724 non-null  object
 3   product_id         144724 non-null  object
 4   product_parent     144724 non-null  int64 
 5   product_title      144724 non-null  object
 6   product_category   144724 non-null  object
 7   star_rating        144724 non-null  int64 
 8   helpful_votes      144724 non-null  int64 
 9   total_votes        144724 non-null  int64 
 10  vine               144724 non-null  object
 11  verified_purchase  144724 non-null  object
 12  review_headline    144722 non-null  object
 13  review_body        144722 non-null  object
 14  review_date        144721 non-null  object
dtypes: int64(5), object(10)
memory usage: 16.6+ MB


### Columns
- **marketplace**: 2 letter country code of the marketplace where the review was written.
- **customer_id**: Random identifier that can be used to aggregate reviews written by a single author.
- **review_id**: The unique ID of the review.
- **productid**: The unique Product ID the review pertains to. In the multilingual dataset the reviews for the same product in different countries can be grouped by the same productid.
- **product_parent**: Random identifier that can be used to aggregate reviews for the same product.
- **product_title**: Title of the product.
- **product_category**: Broad product category that can be used to group reviews (also used to group the dataset into coherent parts).
- **star_rating**: The 1-5 star rating of the review.
- **helpful_votes**: Number of helpful votes.
- **total_votes**: Number of total votes the review received.
- **vine**: Review was written as part of the Vine program.
- **verified_purchase**: The review is on a verified purchase.
- **review_headline**: The title of the review.
- **review_body**: The review text.
- **review_date**: The date the review was written.

In [3]:
df_raw['marketplace'].unique()

array(['US'], dtype=object)

All reviews in dataset are from US customers

In [4]:
df_raw['customer_id'].nunique()

112891

112891 different customers 

In [5]:
df_raw['review_id'].nunique()

144724

Seems like there are no duplicate reviews in the set

In [6]:
df_raw['product_id'].nunique()

7939

7939 different products, but not all video games, found "Playstation Plus Subscription" in row 3, so all digitally sold products related video games, like subscriptions, seem to be included in the data set, will further look for other kind of product other than games and drop those.

In [7]:
df_raw['product_parent'].nunique()

7755

- **product_parent**: Random identifier that can be used to aggregate reviews for the same product.

seems like just another id but we have about 200 less data points in this column than product id

In [8]:
df_raw['product_title'].nunique()

6946

If i have 6946 different titles, i would expect the same amount of product_id, wonder if by droping everything that is not a game, like substriptions, would fix this discrepancy.

In [9]:
df_raw.iloc[3]['product_title']

'Playstation Plus Subscription'

It doesnt specify if the subscription is for 1, 3 or 12 months

<br>

Below i used key words to look for any products that is not a game
e.g
- Subscription
- Month
- Membership
- Card
- Network



In [51]:
def look_keyword(data, strg):
    content = data[data['product_title'].str.contains(strg)]
    print(f'Number of reviews for "{strg}" items: {len(content)} \n')
    
look_keyword(df_raw, "Subscription")    
look_keyword(df_raw, "Month")
look_keyword(df_raw, "Membership")
look_keyword(df_raw, "Card")
look_keyword(df_raw, "Network")

Number of reviews for "Subscription" items: 12137 

Number of reviews for "Month" items: 630 

Number of reviews for "Membership" items: 275 

Number of reviews for "Card" items: 21173 

Number of reviews for "Network" items: 13613 



In [11]:
df_raw['star_rating'].unique()
#games were rated in a scale of 1 to 5 stars

array([2, 5, 4, 1, 3])

In [12]:
df_raw['review_date'] = pd.to_datetime(df_raw['review_date'])
print(f'All reviews date back from {df_raw.review_date.min().year} to {df_raw.review_date.max().year}')

All reviews date back from 2006 to 2015


## Data Preparation

In [13]:
#Droping all rows that have strings: Subscription, Month, Membership,Network on the product_title column
df_raw_noSub = df_raw[df_raw["product_title"].str.contains("Subscription|Month|month|Membership|Network|Credit|Season Pass|Xbox Music Pass|Virtual Currency") == False]


Season Pass, Xbox Music Pass and Virtual Currency are some of the other kind of items on the dataset, i simply added them on the line above as i kept finding more during the process

For titles that have string 'Card' in the title, i have to be more careful since there are some game that include that on their name

In [60]:
def show_keyword(data, strg):
    """
    func just to show examples of titles with given string,
    to find any non-game items in the set
    """
    content = data[data['product_title'].str.contains(strg)]
    return content['product_title'].unique()[:10]
    
    
show_keyword(df_raw_noSub, 'Card')

array(['Xbox Live Gift Card', 'Xbox 360 Live Points Card',
       'Grand Theft Auto V Cash Cards', 'Hoyle Card Games 2012 AMR',
       'Final Fantasy XIV Online: 60 Day Time Card [Online Game Code]',
       'Hoyle Card Games  [Download]',
       'Xbox $5 Gift Card - Xbox 360 Digital Code',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Xbox $15 Gift Card (Call of Duty Ghosts:\xa0Onslaught DLC) - Xbox 360 Digital Code'],
      dtype=object)

First i will drop rows where 'Gift Card' is found, then i will proceed with other common combinations of words like 'Xbox Live' and Live Points Card

In [62]:
df_raw_noSub2 = df_raw_noSub[df_raw_noSub["product_title"].str.contains("Gift Card|Xbox Live|Live Points Card|Xbox Live Gift Card|Grand Theft Auto V Cash Cards|60 Day Time Card") == False]

show_keyword(df_raw_noSub2, 'Card')

array(['Hoyle Card Games 2012 AMR', 'Hoyle Card Games  [Download]',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Tripeaks Solitaire Multi (Tripeaks with Multiple Card-Sets and Multiple Layouts) [Download]',
       'Reel Deal Card Games 2011 [Download]',
       '2,013 Card, Mahjongg & Solitaire Games [Download]',
       'Hoyle Card Games [Mac Download]', 'Five Card Deluxe [Download]',
       "King's Collection: 6 Classic Card Games"], dtype=object)

Seems like all items left that include "card" in the title are actual games

<br>

Lets look for DLCs

In [63]:
DLC = df_raw_noSub2[df_raw_noSub2['product_title'].str.contains('DLC')]
print(f"Total number of DLCs: {DLC['product_title'].nunique()}")
print(f'Total number of DLC reviews:{len(DLC)}')

Total number of DLCs: 212
Total number of DLCs reviews:1698


We have 212 different DLCs, and 1698 reviews, an original copy of the first game is required to be able to run a DLC, I will drop these too.

In [64]:
df_raw_noSub_no_DLC = df_raw_noSub2[df_raw_noSub2["product_title"].str.contains("DLC|Pack") == False]

Quantity for product id and product title still doesn't match

<br>

Some games include a substring next to the title, it describes the way buyers could access the game, for example:

`18 Wheels of Steel American Long Haul [Download]`

`18 Wheels of Steel American Long Haul [Online Game Code]`

Definitely the same game, just different "delivery" method.


In [65]:
df_raw_noSub_no_DLC[df_raw_noSub_no_DLC['product_title'].str.contains('18 Wheels of Steel American Long Haul')].head(3)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
39224,US,36420328,R2LOZS1XJTWZNN,B004GHNG0E,614285704,18 Wheels of Steel American Long Haul [Download],Digital_Video_Games,4,0,0,N,Y,Four Stars,ok,2014-10-15
50242,US,17845931,R2TJLHZ9X7EG83,B004GHNG0E,614285704,18 Wheels of Steel American Long Haul [Download],Digital_Video_Games,4,0,0,N,Y,Four Stars,Like driving trucks.,2014-07-14
74152,US,21374226,R388BHLY4GIU5E,B00CLVZGPK,775610170,18 Wheels of Steel American Long Haul [Online ...,Digital_Video_Games,1,1,1,N,Y,"Doesn't work, no manufacturer support",The code I got does not work. Amazon referred ...,2013-12-29



<br>

The `[Download]` edition of the game, on the second row above, has a different `product_id` than its `[Online Game Code]` edition (on the 3rd row above), it's the same game but taged with different ID, `product_parent` number is also different

<br>

I find it easier to modify the name than the id number, i need to group those reviews togeter, the delivery method doesnt change the software performance, so the review for a specific version is still relevant for a different one.

<br>

At first sight, `[Instant Access]`, `[Download]`, `[Digital Code]`, `[Game Connect]` and `[Online Game Code]` seem to be the most frequent tags at the end of the games titles, if i delete those out of the title name, i could use them for the rec system instead of the id, which was my first option

In [69]:
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['product_title'].str.replace(" \[Instant Access\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Download\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Game Connect\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Online Game Code\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Digital Code\]", "")

<ipython-input-69-de3b04f97402>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['product_title'].str.replace(" \[Instant Access\]", "")
<ipython-input-69-de3b04f97402>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Download\]", "")
<ipython-input-69-de3b04f97402>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

The first title (on index 0) has 'Xbox One Digital Code' in its name, will check if there are more games with this same substring, or even for a different platform other than xbox

In [67]:
#below i deleted the susbstring using same method as before
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox One Digital Code", "")

#found it for the 360 console too
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox 360 Digital Code", "")

#found ' - PS4', ' - PS3', ' - PS Vita / PS4 / PS3', ' - PS Vita'
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS Vita / PS4 / PS3", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS Vita", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS3", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS4", "")

<ipython-input-67-00b8bc71ca71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox One Digital Code", "")
<ipython-input-67-00b8bc71ca71>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox 360 Digital Code", "")


### Clean DF

In [23]:
#New frame with cleaned data and only columns needed
Df_clean = df_raw_noSub_no_DLC[['customer_id', 'clean_title', 'star_rating']]
#reseting index
Df_clean.reset_index(drop=True, inplace=True)

print(f'Total number of customers in dataset:{Df_clean.customer_id.nunique()}')
print(f'Total number of games in dataset:{Df_clean.clean_title.nunique()}')

Total number of customers in dataset:76941
Total number of games in dataset:6001


In [24]:
# Subsetting to remove the customers with less than 2 reviews
Df_clean_subset = Df_clean[Df_clean['customer_id'].map(Df_clean['customer_id'].value_counts()) > 1]

print(f'Total number of customers in dataset:{Df_clean_subset.customer_id.nunique()}')
print(f'Total number of games in dataset:{Df_clean_subset.clean_title.nunique()}')
print((f'Total number of ratings in dataset:{len(Df_clean_subset)}'))

Total number of customers in dataset:11133
Total number of games in dataset:4756
Total number of ratings in dataset:35362


In [25]:
#Saving as new file
Df_clean_subset.to_csv('./data/GameRatings.csv', index=False)
Ratings = pd.read_csv('./data/GameRatings.csv')

### Preparing for modeling

In [26]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [27]:
reader = Reader(line_format='user item rating',
                sep=',', 
                rating_scale=(1, 5))

data = Dataset.load_from_df(Ratings[['customer_id', 'clean_title', 'star_rating']], reader=reader)

In [28]:
trainset, testset = train_test_split(data, test_size=0.2)

print('Number of users: ', trainset.n_users)
print('Number of items: ', trainset.n_items)
print('Number of ratings: ', trainset.n_ratings)

Number of users:  10829
Number of items:  4424
Number of ratings:  28289


In [29]:
#for looking back into de meaning of each id
trainset_iids = list(trainset.all_items())
iid_converter = lambda x: trainset.to_raw_iid(x)
trainset_raw_iids = list(map(iid_converter, trainset_iids))

# Modeling

I will be using scikit-surprise models to built the recomender system, Using collaborative-filtering, i will first train a model with a memory-based method; later i will explore and perform gridsearch with a model-based approach

## KNN Baseline Model

In [84]:
import surprise
from surprise.prediction_algorithms import *
from surprise import accuracy

KNNbaseline_model = KNNBaseline()
KNNbaseline_model.fit(trainset)

#getting predictions using testset
KNNb_predictions = KNNbaseline_model.test(testset)

# Root Mean Square Error and Mean Absolute Error
accuracy.rmse(KNNb_predictions)
accuracy.mae(KNNb_predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3764
MAE:  1.0816


1.0815825703187985

**Cross validation**

In [77]:
from surprise.model_selection import cross_validate

KNNb_cv = cross_validate(algo = KNNbaseline_model, 
                         data = data, 
                         measures=['RMSE'], 
                         cv=5, 
                         n_jobs= -2,
                         return_train_measures=True, 
                        verbose=True)

Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3752  1.3637  1.4010  1.3841  1.3857  1.3819  0.0123  
RMSE (trainset)   0.4169  0.4172  0.4159  0.4160  0.4163  0.4165  0.0005  
Fit time          7.00    7.09    7.00    6.78    6.75    6.92    0.14    
Test time         0.26    0.26    0.26    0.27    0.27    0.26    0.01    


In [79]:
KNNb_cv['test_rmse'].mean()

1.381937964988524

## SVD Simple model

In [83]:
from surprise.prediction_algorithms import SVD

SVD_model = SVD(random_state=1)
SVD_model.fit(trainset)

#getting predictions using testset
SVD_predictions = SVD_model.test(testset)

# Root Mean Square Error and Mean Absolute Error
accuracy.rmse(SVD_predictions)
accuracy.mae(SVD_predictions)

RMSE: 1.3008
MAE:  1.0567


1.056711892024892

RMSE slightly  lower than KNN baseline,

**Cross validation**

In [78]:
SVD_cv = cross_validate(algo = SVD_model, 
                        data = data, 
                        measures=['RMSE'], 
                        cv=5, 
                        n_jobs=-2, 
                        verbose=True, 
                        return_train_measures=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3064  1.2975  1.3307  1.2826  1.3016  1.3038  0.0156  
RMSE (trainset)   0.8788  0.8788  0.8712  0.8815  0.8765  0.8774  0.0035  
Fit time          1.37    1.35    1.38    1.54    1.37    1.40    0.07    
Test time         0.03    0.03    0.03    0.03    0.03    0.03    0.00    


In [80]:
SVD_cv['test_rmse'].mean()

1.3037753163631471

### SVD Gridsearch

In [89]:
from surprise.model_selection import GridSearchCV

# Setting hyperparameters around default values
SVD_grid = {'n_factors':[50, 100, 120],
           'n_epochs':[10, 20, 40],
           'lr_all':[0.002, 0.005, 0.05],
           'reg_all':[0.02, 0.1], 
           'biased':[True, False]}

SVD_gs = GridSearchCV(algo_class = SVD, 
                      param_grid = SVD_grid, 
                      measures=['rmse'], 
                      n_jobs = -2, 
                      joblib_verbose = 5)

SVD_gs.fit(data)
SVD_gs.best_params

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-2)]: Done 274 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-2)]: Done 436 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-2)]: Done 540 out of 540 | elapsed:  2.2min finished


{'rmse': {'n_factors': 100,
  'n_epochs': 20,
  'lr_all': 0.05,
  'reg_all': 0.1,
  'biased': True}}

In [90]:
# using best parameters in new model and only fiting train set
svd_1 = SVD(lr_all=0.05, reg_all= .01)

svd_1.fit(trainset)

#getting predictions using testset
SVD_pred_1 = svd_1.test(testset)

# Root Mean Square Error and Mean Absolute Error
accuracy.rmse(SVD_pred_1)
accuracy.mae(SVD_pred_1)

RMSE: 1.2748
MAE:  1.0134


1.0133878649254648

Gridsearch did improve RMSE from baseline model, biased, n_factors and n_epochs stayed in default values while lr_all and reg_all where pick with hightes value given.

### SVD Gridsearch 2

In [92]:
# Setting hyperparameters around default values
SVD_grid_2 = {'n_factors':[50, 100, 120],
           'n_epochs':[10, 20, 40],
           'lr_all':[0.005, 0.05, 0.08],
           'reg_all':[0.02, 0.1, 0.3, 0.5]}

SVD_gs_2 = GridSearchCV(algo_class = SVD, 
                      param_grid = SVD_grid_2, 
                      measures=['rmse'], 
                      n_jobs = -2, 
                      joblib_verbose = 5)

SVD_gs_2.fit(data)
SVD_gs_2.best_params

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-2)]: Done 274 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-2)]: Done 436 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-2)]: Done 540 out of 540 | elapsed:  2.2min finished


{'rmse': {'n_factors': 100, 'n_epochs': 10, 'lr_all': 0.05, 'reg_all': 0.3}}

In [93]:
# using best parameters in new model and only fiting train set
svd_2 = SVD(n_epochs= 10, lr_all=0.05, reg_all= 0.3)

svd_2.fit(trainset)

#getting predictions using testset
SVD_pred_2 = svd_2.test(testset)

# Root Mean Square Error and Mean Absolute Error
accuracy.rmse(SVD_pred_2)
accuracy.mae(SVD_pred_2)

RMSE: 1.2591
MAE:  1.0114


1.011404739566474

### SVD Gridsearch 3

In [94]:
# Setting hyperparameters around default values
SVD_grid_3 = {'n_factors':[80, 100, 120],
           'n_epochs':[5, 10, 15],
           'lr_all':[0.01, 0.05, 0.08],
           'reg_all':[0.1, 0.3, 0.5]}

SVD_gs_3 = GridSearchCV(algo_class = SVD, 
                      param_grid = SVD_grid_3, 
                      measures=['rmse'], 
                      n_jobs = -2, 
                      joblib_verbose = 5)

SVD_gs_3.fit(data)
SVD_gs_3.best_params

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-2)]: Done 274 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-2)]: Done 405 out of 405 | elapsed:   49.7s finished


{'rmse': {'n_factors': 80, 'n_epochs': 10, 'lr_all': 0.05, 'reg_all': 0.3}}

In [95]:
# using best parameters in new model and only fiting train set
svd_3 = SVD(n_factors=80, n_epochs= 10, lr_all=0.05, reg_all= 0.3)

svd_3.fit(trainset)

#getting predictions using testset
SVD_pred_3 = svd_3.test(testset)

# Root Mean Square Error and Mean Absolute Error
accuracy.rmse(SVD_pred_3)
accuracy.mae(SVD_pred_3)

RMSE: 1.2561
MAE:  1.0068


1.0067608879084728

Decrease on RMSE from previous gridsearch was minimal, will move on and SVD++

## SVD++

In [ ]:
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

## Evaluation

## Deployment

## metadata

!ls data


import gzip

vec_tfidf

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF('./data/meta_Video_Games.json.gz')